<h1> Progetto DataManagement </h1>

<h2>1) Creazione del database e tabelle </h2>

In [8]:
# Importo la libreria per poter effettuare la connessione
import sqlite3

<b> Creazione e connessione con il database </b>

In [9]:
conn = sqlite3.connect('DataManagement.db')
conn.execute('PRAGMA foreign_keys = ON')
cursor = conn.cursor()

<b> Creazione tabella comuni </b>

In [16]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS comuni (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nome TEXT,
        comune_appartenenza TEXT,
        comuneId INTEGER,
        `vendita_minima(€/mese)` DECIMAL(10,2),
        `vendita_media(€/mese)` DECIMAL(10,2),
        `vendita_massima(€/mese)` DECIMAL(10,2),
        `affitto_minimo(€/mese)` DECIMAL(10,2),
        `affitto_medio(€/mese)` DECIMAL(10,2),
        `affitto_massimo(€/mese)` DECIMAL(10,2),
        popolazione number(10,2),      
        `densita_popolazione(ab./km²)` number(10,2),
        `superficie(km²)` number(10,2),
        famiglie number(10,2),
        `maschi(%)` number(10,2),
        `femmine(%)` number(10,2),
        `stranieri(%)` number(10,2),
        `eta_media(anni)` number(10,2),
        FOREIGN KEY (comuneId) REFERENCES comuni(id) ON DELETE CASCADE
    )
''')

<b> Creazione tabella affitti </b>

In [17]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS affitti (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nome TEXT,
        comuneId INTEGER,
        contratto TEXT,
        tipologia TEXT,
        `superficie(m²)` DECIMAL(10,2),
        locali INTEGER,
        locali_descrizione TEXT,
        piani TEXT,
        indirizzo TEXT,
        latitudine TEXT,
        longitudine TEXT,
        stazioneId INTEGER,
        `stazione_distanza(metri)` INTEGER,
        `stazione_distanza(minuti)` INTEGER,
        `prezzo(€/mese)` DECIMAL(10,2),
        `prezzo_minimo(€/mese)` DECIMAL(10,2),
        `prezzo_massimo(€/mese)` DECIMAL(10,2),
        box_auto INTEGER,
        `giardino(m²)` INTEGER,
        balcone INTEGER,
        ascensore INTEGER,
        fumatori INTEGER,
        animali INTEGER,
        url_annuncio TEXT,    
        FOREIGN KEY (comuneId) REFERENCES comuni(id) ON DELETE CASCADE
        FOREIGN KEY (stazioneId) REFERENCES mezzi_pubblici(id) ON DELETE CASCADE
    )
''')

<b> Creazione tabella luoghi_interesse </b>

In [18]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS luoghi_interesse (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nome TEXT,
        descrizione TEXT,
        latitudine TEXT,
        longitudine TEXT,
        tempo_arrivo TEXT,
        week_day TEXT
    )
''')

<b> Creazione tabella mezzi_pubblici </b>

In [19]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS mezzi_pubblici (
        id INTEGER PRIMARY KEY,
        nome TEXT,
        areaSTIBM TEXT,
        latitudine TEXT,
        longitudine TEXT,
        tipo_mezzo TEXT
    )
''')

<b> Creazione tabella mezzi_luogo </b>

In [39]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS mezzi_luogo (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        id_mezzi_pubblici INTEGER,
        id_luogo_interesse INTEGER,
        `distanza(minuti)` INTEGER,
        `distanza(km)` INTEGER,
        FOREIGN KEY (id_mezzi_pubblici) REFERENCES mezzi_pubblici(id) ON DELETE CASCADE
        FOREIGN KEY (id_luogo_interesse) REFERENCES luoghi_interesse(id) ON DELETE CASCADE
    )
''')

In [23]:
# Effettuo una commit delle tabelle create
conn.commit()

<b> Cancello le tabelle create </b>

In [6]:
cursor.execute("drop table comuni")

In [7]:
cursor.execute("drop table affitti")

In [12]:
cursor.execute("drop table luoghi_interesse")

OperationalError: no such table: luoghi_interesse

In [13]:
cursor.execute("drop table mezzi_pubblici")

In [38]:
cursor.execute("drop table mezzi_luogo")

In [21]:
conn.commit()

<h2> 2) Utilizzo le API per prelevare i dati sui mezzi pubblici</h2>

In [12]:
import requests
import json
import pandas as pd
from pandas import json_normalize
from numpy import isnan
from numpy import NaN

In [13]:
# data request to Overpass API using Overpass QL 
url = "https://maps.mail.ru/osm/tools/overpass/api/interpreter"
query = """
        [out:json];
        area[name="Milano"];
        nwr[railway=station][station=subway](area);
        out center;
        """
response = requests.get(url, params={'data': query})
data = response.json()

In [14]:
# conversion to tabular
metro_df = json_normalize(data['elements'])

# feature selection
feature = ['tags.name','tags.description','lon','lat']
metro_df = metro_df[feature]

# strip 'Area STIBM:' from tags.description
for i in range(0, metro_df.shape[0]):
    if metro_df['tags.description'][i] is not NaN:
        metro_df['tags.description'][i] = metro_df['tags.description'][i].replace('Area STIBM: ', '')

# renaming columns
metro_df.rename(columns={"tags.name":"name","tags.description":"Area STIBM"}, inplace=True)

# drop duplicate (stations where metro lines meet are duplicated) 
metro_df.drop_duplicates(subset=['name'], ignore_index=True, inplace=True)

# drop station with unknown location
metro_df.drop(metro_df[isnan(metro_df['lon']) & isnan(metro_df['lat'])].index, inplace = True)

<ipython-input-14-aaef25273577>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metro_df['tags.description'][i] = metro_df['tags.description'][i].replace('Area STIBM: ', '')


In [15]:
url = "https://maps.mail.ru/osm/tools/overpass/api/interpreter"
query = """
        [out:json];
        area[name="Milano"];
        nwr[railway=station][station!=subway](area);
        out center;
        """
response = requests.get(url, params={'data': query})
data = response.json()

In [16]:
# conversion to tabular
train_df = json_normalize(data['elements'])

# feature selection
feature = ['tags.name','tags.description','lon','lat']
train_df = train_df[feature]

# strip 'Area STIBM:' from tags.description
for i in range(0, train_df.shape[0]):
    if train_df['tags.description'][i] is not NaN:
        train_df['tags.description'][i] = train_df['tags.description'][i].replace('Area STIBM: ', '')

# renaming columns
train_df.rename(columns={"tags.name":"name","tags.description":"Area STIBM"}, inplace=True)

# drop duplicate (stations where metro lines meet are duplicated) 
train_df.drop_duplicates(subset=['name'], ignore_index=True, inplace=True)

# drop station with unknown location
train_df.drop(train_df[isnan(train_df['lon']) & isnan(train_df['lat'])].index, inplace = True)

<ipython-input-16-9ffab8ca2c1a>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['tags.description'][i] = train_df['tags.description'][i].replace('Area STIBM: ', '')


In [17]:
# join train and metro dataframes into one
train_df['station_type']='treno'
metro_df['station_type']='metro'
public_transport_df = pd.concat([train_df,metro_df], ignore_index=True)
public_transport_df.index += 1 

In [18]:
public_transport_df

,name,Area STIBM,lon,lat,station_type
1,Milano Cadorna,Mi1,9.175523,45.468405,treno
2,Milano Porta Garibaldi (superficie),Mi1,9.187309,45.484530,treno
3,Milano San Cristoforo,NaN,9.130128,45.442302,treno
4,Milano Greco Pirelli,Mi1,9.214188,45.512889,treno
5,Rho,Mi4,9.043565,45.524102,treno
...,...,...,...,...,...
173,Isola,Mi1,9.191294,45.487612,metro
174,Dateo,NaN,9.218447,45.468001,metro
175,Argonne,NaN,9.231908,45.468083,metro
176,Susa,NaN,9.224874,45.468067,metro


<b>Creo un file csv del mio dataFrame </b>

In [29]:
public_transport_df.to_csv('mezzi_pubblici.csv', index=True, header=False)

<h2> 3) Creare un dataframe coi luoghi d'interesse da un csv creato in precedenza </h2>

In [22]:
luoghi_interesse = pd.read_csv('luoghi_interesse.csv')

In [23]:
luoghi_interesse.index += 1 

In [24]:
luoghi_interesse

,nome,descrizione,latitudine,longitudine,tempo_arrivo,week_day
1,Università degli Studi di Milano Bicocca Piazz...,Bicocca,45.518316,9.213788,8:30,VERO
2,Politecnico di Milano Piazza Leonardo da Vinci 32,Polimi Piola,45.478188,9.227235,8:30,VERO
3,Politecnico di Milano - Campus Bovisa La Masa ...,Polimi Bovisa,45.502873,9.156420,8:30,VERO
4,Università degli Studi di Milano Via Festa del...,Statale sede principale,45.460395,9.194151,8:30,VERO
5,Arco della Pace Piazza Sempione,Arco della pace,45.475715,9.172451,20:00,FALSO
6,Piazza del Duomo- Milano,Piazza Duomo,45.464230,9.189630,15:00,FALSO
7,Stadio San Siro Piazzale Angelo Moratti,Stadio San Siro,45.478061,9.123959,20:00,FALSO
8,Pinacoteca di Brera Via Brera 28,Pinacoteca di Brera,45.471954,9.187814,15:00,FALSO
9,Darsena del Naviglio,Navigli,45.452553,9.178126,20:00,FALSO
10,Alcatraz Via Valtellina,Alcatraz,45.494696,9.182647,20:00,FALSO


<h2> 4) Popolo le tabelle con i dati dei csv </h2>

<b>Inserisco i dati dei mezzi pubblici in tabella </b>

In [34]:
import csv

In [35]:
with open('mezzi_pubblici.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        id_mezzo = int(row[0])
        nome = str(row[1]).replace("'", " ").replace("Ã", "à").replace('â€™', " ").replace('Âº', '').replace('à²','ò')
        area = str(row[2])
        latitudine = str(row[3])
        longitudine = str(row[4])
        tipo = str(row[5]).replace(';', '')
        # Costruisci la query SQL per l'inserimento
        insert_query = f"INSERT INTO mezzi_pubblici (id, nome, areaSTIBM, latitudine, longitudine, tipo_mezzo ) VALUES ('{id_mezzo}','{nome}','{area}','{latitudine}','{longitudine}','{tipo}')"
        # Esegui la query
        cursor.execute(insert_query)
        conn.commit()

<b>Inserisco i dati dei luoghi di interesse in tabella </b>

In [36]:
cont = 0
with open('luoghi_interesse.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        nome = str(row[0]).replace("'", " ").replace("Ã", "à")
        descrizione = str(row[1])
        latitudine = str(row[2])
        longitudine = str(row[3])
        tempo = str(row[4])
        week_day = str(row[5])
        # Costruisci la query SQL per l'inserimento
        if (cont > 0):
            insert_query = f"INSERT INTO luoghi_interesse (nome, descrizione, latitudine, longitudine, tempo_arrivo, week_day) VALUES ('{nome}','{descrizione}','{latitudine}','{longitudine}','{tempo}','{week_day}')"
            cursor.execute(insert_query)
            conn.commit()
        cont = cont+1

<h2> 4) Utilizzo Selenium per trovare distanza e tempo tra luoghi e stazione</h2>

In [30]:
##### import required modules #####
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
from tqdm import tqdm
from selenium.webdriver.common.keys import Keys

In [31]:
##### define functions #####

## set_up ## 
# opens and sets up the google maps page
# arrive_time: set the arrive time; format 'hh:mm'
# week_day: set the arrive day; accepted value = True (day is set to monday), False (day is set to saturday)

def set_up(arrive_time,week_day):

    # assign url in the webdriver object
    global driver 
    driver = webdriver.Chrome()
    driver.get("https://www.google.it/maps/dir/@45.5038144,9.4419926,15z?entry=ttu")
    sleep(2)

    # Accept cookie policy
    Submit = driver.find_elements(By.TAG_NAME, "button")
    Submit[1].click()
    sleep(2)

    # initialize the page with an arbitrary search
    searchplace("metro precotto","Piazza della Scienza, 20126 Milano MI")
    sleep(5)

    # select "arrive by" mode
    driver.find_element(By.XPATH,'//*[@id=":3"]').click()
    driver.find_element(By.XPATH,'//*[@id=":2"]').click()
    sleep(1)

    # select arrive time
    path ='//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div/span[1]/input'
    driver.find_element(By.XPATH,path).clear()
    driver.find_element(By.XPATH,path).send_keys(arrive_time)
    sleep(1)

    # select arrive day
    path_testo ='//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div/span[2]/span[1]'
    path_bottone = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div/span[2]/span[2]/button[2]'
    giorno = driver.find_element(By.XPATH,path_testo).text

    if week_day:
        target = 'lun'
    else:
        target = 'sab'
            
    while (not target in giorno):
        driver.find_element(By.XPATH,path_bottone).click()
        giorno = driver.find_element(By.XPATH,path_testo).text
        sleep(1)

        
## searchplace ## 
# inputs starting point (A) and destination (B)

def searchplace(A,B):
    Place = driver.find_elements(By.CLASS_NAME,"tactile-searchbox-input")
    Place[0].clear()
    Place[0].send_keys(A)
    Place[1].clear()
    Place[1].send_keys(B)
    Place[1].send_keys(Keys.RETURN)
    
    
## extract_time ##
# extracts the time needed to get from A to B by public transport

def extract_time():
    # read time from the web page
    path_tempo = '//*[@id="omnibox-directions"]/div/div[2]/div/div/div/div[3]/button/div[1]'
    tempo = driver.find_element(By.XPATH,path_tempo).text
    # convert time format in number of minutes 
    if tempo!='':
        numbers = [int(s) for s in tempo.split() if s.isdigit()]
        if len(numbers)==2:
            time = numbers[0]*60 + numbers[1]
        else:
            time = numbers[0] 
    else:
        time = 0
    
    return int(time) 


## time_table ##
# compute the time needed to get from each metro and train station to a given point of interest
# point of interest: a string that google maps interprets as the desired place
# delay: sets the timing of the loop, adjust for internet speed
# arrive_time: set the arrive time; format 'hh:mm'
# week_day: set the arrive day; accepted value = True(day is set to monday), False(day is set to saturday)

def time_table(point_of_intrest, arrive_time, week_day ,delay=5):
    set_up(arrive_time,week_day)
    time_list = []
    station_id = []
    
    # loops over stations and retrives the needed time storing it along
    # with the station index in time_list an station_id respectively
    for index,row in tqdm(public_transport_df.iterrows(),total=public_transport_df.shape[0], desc='Progress'):
        station = row['name'] + ' ' + row['station_type']
        searchplace(station,point_of_intrest)
        sleep(delay)
        tempo = extract_time()
        time_list.append(tempo)
        station_id.append(index)
        
    # creates a dataframe containing station_id and time_list   
    df = pd.DataFrame(data={'station_id': station_id, 'time': time_list})
    return df

<b> Faccio partire i cicli iterativi con selenium per recuperare la relazione </b>

In [32]:
import geopy.distance
def air_distance(lon1, lat1, lon2, lat2):
    coords_1 = (lon1,lat1)
    coords_2 = (lon2,lat2)
    return geopy.distance.geodesic(coords_1, coords_2).km

In [33]:
import re

In [40]:
for index, row in luoghi_interesse.iterrows():
    id_luogo = index
    df = time_table(row['nome'], row['tempo_arrivo'], row['week_day'], delay=3)
    for index1, row1 in df.iterrows():
        id_station = row1['station_id']
        tempo = row1['time']
        distanza = round(air_distance(row['longitudine'], row['latitudine'], public_transport_df.iloc[index1]['lon'], public_transport_df.iloc[index1]['lat']),2) 
        tempo = int(tempo) if (tempo!=0) else int(distanza*12)
        cursor.execute(f"INSERT INTO mezzi_luogo(id_mezzi_pubblici, id_luogo_interesse, `distanza(minuti)`, `distanza(km)`) VALUES('{id_station}','{id_luogo}','{tempo}','{distanza}')")
        conn.commit()

Progress: 100%|██████████████████████████████████████████████████████████████████████| 177/177 [10:16<00:00,  3.48s/it]


<h2> 5) Scrapyng dati e storage su database </h2>

<b>Importo le librerie necessarie per lo scraping</b>

In [42]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from datetime import datetime as dt
import logging
import json

<b>Inserisco l'indirizzo url per effettuare lo scraping</b>

In [43]:
url = "https://www.immobiliare.it/mercato-immobiliare/lombardia/milano-provincia/" 
response = requests.get(url)

In [44]:
soup = BeautifulSoup(response.content, 'html.parser')

<b>Recupero il div con l'elenco dei link di tutte le province</b>

In [45]:
div_link_comuni = soup.find_all('a', {"class": "nd-table__url"}, href=True)

<b>Creo una lista coi singoli link delle pagine da dove fare scraping</b>

In [46]:
lista_link_comuni = []
for link in div_link_comuni:
    lista_link_comuni.append(link.get("href"))

<b>Definisco la funzione per recuperare la stazione piu vicina</b>

In [47]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [48]:
import geopy.distance
 # long,lat of the house; df: metro_df or train_df
def nearest_point(long, lat):
    coords_1 = (long,lat)
    distance=[]
    for i in range(1, public_transport_df.shape[0]):
        coords_2 = (public_transport_df.lon[i], public_transport_df.lat[i])
        distance.append(geopy.distance.geodesic(coords_1, coords_2).km)
    return({'index':distance.index(min(distance)) + 1, 'distance':round(min(distance),1)})

<b>Definisco la funzione per recuperare le zone o frazioni di un comune</b>

In [50]:
def recupera_zone(soup, nome_comune_appartenenza, insertQueryComuni, insertQueryAffitti):
    div_link_zone = soup.find_all('a', {"class": "nd-table__url"}, href=True)
    lista_link_zone = []

    for link in div_link_zone:
        lista_link_zone.append(link.get("href"))
    
    
    for link in lista_link_zone:
        response = requests.get(link)
        soup = BeautifulSoup(response.content, 'html.parser')

        # recupero il nome del comune
        nome = str(soup.find_all("li", attrs = {"class": "in-breadcrumbsLite__item nd-list__item"})[4].get_text()).replace('\t', '').replace('\n', '')

        # recupero le due strutture contenenti le vendite e affitti minimi e massimi, vendite e affitti medi
        vendita_affitto_medio = soup.find_all("p", attrs={"class": "cg-buildingPricesStats__highlighted-text"})

        vendita_affitto = soup.find_all("p", attrs={"class": "cg-buildingPricesStats__highlighted-subtext"})

        # salvo le due stringhe contenenti vendite e affitti minimi e massimi   
        stringa_vendite = str(vendita_affitto[0].get_text())
        stringa_affitti = str(vendita_affitto[1].get_text())

        # salvo nelle variabili

        vendita_media = float(str(re.findall(r'\d+[\.,]?\d+', str(vendita_affitto_medio[0].get_text()))[0]).replace('.', '').replace(',', '.'))
        affitto_medio = float(str(re.findall(r'\d+[\.,]?\d+', str(vendita_affitto_medio[1].get_text()))[0]).replace('.', '').replace(',', '.'))

        if(stringa_vendite == '-'):
            vendita_minima = 0
            vendita_massima = 0
        else:
            vendita_minima = float(str(re.findall(r'\d+[\.,]?\d+', stringa_vendite)[0]).replace('.', '').replace(',', '.'))
            vendita_massima = float(str(re.findall(r'\d+[\.,]?\d+', stringa_vendite)[1]).replace('.', '').replace(',', '.'))

        if(stringa_affitti == '-'):
            affitto_minimo = 0
            affitto_massimo = 0
        else:
            affitto_minimo = float(str(re.findall(r'\d+[\.,]?\d+', stringa_affitti)[0]).replace('.', '').replace(',', '.'))
            affitto_massimo = float(str(re.findall(r'\d+[\.,]?\d+', stringa_affitti)[1]).replace('.', '').replace(',', '.'))

       
        # recupero div con abitazioni, popolazione, eta media e reddito
        superficie = 0
        densita = 0
        popolazione = 0
        famiglie = 0
        maschi = 0
        femmine = 0
        stranieri = 0
        eta = 0
    

        # creo un nuovo record per la tabella comuni e lo inserisco
        
        # recupero il comuneId di appartenenza
        cursor.execute("SELECT id FROM comuni WHERE nome = ?", (nome_comune_appartenenza,))
        comuneId_appartenenza = cursor.fetchone()[0]
        
         # faccio la insert

        logging.info("Sto inserendo il " + nome + " del comune di " + nome_comune_appartenenza + " nella tabella COMUNI")
        cursor.execute(insertQueryComuni, 
                           (nome, nome_comune_appartenenza, comuneId_appartenenza, vendita_minima, vendita_media, vendita_massima, affitto_minimo, affitto_medio, affitto_massimo, popolazione, densita, superficie, famiglie, maschi, femmine, stranieri, eta))

        conn.commit()
        
        # richiamo le funzioni per recuperare gli affitti
        link_affitto = soup.find_all('a', {"class": "cg-realEstateAdsCounters__link"}, href=True)[1]
        link_affitto = str(link_affitto['href'])
        
        # recupero id del comune o frazione
        
        cursor.execute("SELECT id FROM comuni WHERE nome = ? AND comuneId = ?", (nome, comuneId_appartenenza))
        comuneId = cursor.fetchone()[0]
        
        # richiamo le 2 funzioni
        recupera_affitti(link_affitto, comuneId, insertQueryAffitti)
        

<b>Definisco la funzione per recuperare gli affitti di un determinato comune</b>

In [51]:
def recupera_affitti(link_affitto, comuneId, insertQueryAffitti):
    
    lista_link_affitti = recupera_all_annunci(link_affitto)
    
    logging.info("Sto inserendo gli annunci nella tabella AFFITTI, sono in totale: " + str(len(lista_link_affitti)) + "\n")
    
    for link in lista_link_affitti:
        
        response = requests.get(link)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # recupero il prezzo, luogo e tipologia di immobile
        div_prezzo = soup.find("li", class_="nd-list__item in-feat__item in-feat__item--main in-detail__mainFeaturesPrice")
        div_prezzo_altro = soup.find("li", class_="nd-list__item in-feat__item in-feat__item--main in-detail__mainFeaturesPrice in-detail__mainFeaturesPrice--interactive")
        
        if(div_prezzo is not None):
            div_prezzo = str(div_prezzo.get_text()) 
            prezzo = re.findall(r'\d+[\.,]?\d+', div_prezzo)
            prezzo = float(str(prezzo[0]).replace('.', '').replace(',', '.')) if(len(prezzo) > 0) else 0
        else:
            div_prezzo = str(div_prezzo_altro.get_text()) if div_prezzo_altro is not None else '0'
            prezzo = re.findall(r'\d+[\.,]?\d+', div_prezzo)
            prezzo = float(str(prezzo[0]).replace('.', '').replace(',', '.')) if (len(prezzo) > 0) else 0
        
        
        # recupero il luogo e la tipologia di immobile
        div_luogo = soup.find("h1", class_="in-titleBlock__title")            
        indirizzo = ' '.join(str(div_luogo.get_text()).split()[1:]) if div_luogo is not None else '-'
        #tipologia = str(str(div_luogo.get_text()).split()[0]) if div_luogo is not None else '-'
        
        
        # recupero tutte le altre informazioni
        
        div_affitti = soup.find_all("dd", class_="in-realEstateFeatures__value")
        if (len(div_affitti) > 4): 
            superficie = re.findall(r'\d+[\.,]?\d+', str(div_affitti[3].get_text()))
            superficie = float(str(superficie[0]).replace('.', '').replace(',', '.')) if (len(superficie) > 0) else 0
            locali_descrizione = str(div_affitti[4].get_text())
            locali = re.findall(r'\d', locali_descrizione)
            locali = int(locali[0]) if (len(locali) > 0) else 0
            piani = str(div_affitti[5].get_text())
        else:
            locali_descrizione = ''
            superficie = 0
            locali = 0
            piani = ''
            
        # recupero tramite lo script le informazioni necessarie
        item_annuncio = soup.find("script", id="__NEXT_DATA__")
        
        if (item_annuncio is not None and len(item_annuncio) > 0):

            item_annuncio = str(item_annuncio).replace('<script id="__NEXT_DATA__" type="application/json">', '').replace('</script>', '')
            json_dict = json.loads(item_annuncio)

            nome = json_dict['props']['pageProps']['detailData']['realEstate']['title']
            contratto = json_dict['props']['pageProps']['detailData']['realEstate']['contractValue']
            latitudine = json_dict['props']['pageProps']['detailData']['realEstate']['properties'][0]['location']['latitude']
            longitudine = json_dict['props']['pageProps']['detailData']['realEstate']['properties'][0]['location']['longitude']
            tipologia = json_dict['props']['pageProps']['detailData']['realEstate']['properties'][0]['typologyValue']
            prezzo_minimo = json_dict['props']['pageProps']['detailData']['trovakasa']['minPrice']
            prezzo_minimo = float(prezzo_minimo) if prezzo_minimo is not None else 0
            prezzo_massimo = json_dict['props']['pageProps']['detailData']['trovakasa']['maxPrice']
            prezzo_massimo = float(prezzo_massimo) if prezzo_massimo is not None else 0
            box_auto = json_dict['props']['pageProps']['detailData']['trovakasa']['boxAutoId']
            box_auto = 0 if box_auto is None else int(box_auto)
            giardino = json_dict['props']['pageProps']['detailData']['trovakasa']['garden']
            giardino = 0 if giardino is None else int(giardino)
            balcone = json_dict['props']['pageProps']['detailData']['trovakasa']['balcony']
            balcone = 0 if balcone is None else 1
            ascensore = json_dict['props']['pageProps']['detailData']['trovakasa']['elevator']
            ascensore = 0 if ascensore is None else int(ascensore)
            fumatori = json_dict['props']['pageProps']['detailData']['trovakasa']['smoker']
            fumatori = 0 if fumatori is None else int(fumatori)
            animali = json_dict['props']['pageProps']['detailData']['trovakasa']['animals']
            animali = 0 if animali is None else int(animali)
        else:
            nome = ''
            contratto = ''
            latitudine = ''
            longitudine = ''
            tipologia = ''
            prezzo_minimo = 0
            prezzo_massimo = 0
            box_auto = 0 
            giardino = 0
            balcone = 0 
            ascensore = 0 
            fumatori = 0
            animali = 0

        # stazione piu vicina implementando le API
        dizionario = nearest_point(longitudine, latitudine)
        stazioneId = int(dizionario['index'])
        stazione_distanza = int(dizionario['distance'] * 1000)
        stazione_tempo = int(dizionario['distance'] * 12)
        
        # controllo che non sia gia salvato l'annuncio 
        cursor.execute("SELECT id FROM affitti WHERE url_annuncio = ?", (link,))
        nonEsiste = True if cursor.fetchone() is None else False
        
        if(nonEsiste):
            # inserisco un nuovo record nella tabella affitti
            cursor.execute(insertQueryAffitti, (nome, comuneId, contratto, tipologia, superficie, locali, locali_descrizione, piani, indirizzo, latitudine, longitudine, stazioneId, stazione_distanza, stazione_tempo, prezzo, prezzo_minimo, prezzo_massimo, box_auto, giardino, balcone, ascensore, fumatori, animali, link))
            conn.commit()
    
        

<b>Definisco la funzione per recuperare tutti gli annunci di affitto di un comune o frazione</b>

In [52]:
def recupera_all_annunci(link_pagina):
    
    #print("Sto recuperando gli annunci...")
    logging.info("Sto recuperando gli annunci...")
    lista_link_annunci = []
    numero_pagina = 1
    annunci = []
    
    while True:
        
        response = requests.get(link_pagina + "?pag=" + str(numero_pagina))
        soup = BeautifulSoup(response.content, 'html.parser')

        pagina_non_esiste = soup.find("div", class_="nd-alert nd-alert--warning in-errorMessage__alert in-errorMessage__title")

        if (pagina_non_esiste is None):

            div_link_annunci = soup.find_all("div", class_="in-card nd-mediaObject nd-mediaObject--colToRow in-realEstateCard in-realEstateCard--interactive in-realEstateListCard")
            
            for link in div_link_annunci:
                link_annuncio = link.find("a", class_="in-card__title")
                if len(link_annuncio) > 0:
                    lista_link_annunci.append(link_annuncio["href"])     
        else:
            break


        numero_pagina = numero_pagina + 1
            
    return lista_link_annunci

<b>Definisco una funzione per visualizzare il tempo necessario allo scraping:</b>

In [53]:
def stampaTempi(ora_inzio, ora_fine):
    differenza = ora_fine - ora_inizio
    ore = differenza.seconds // 3600
    minuti = (differenza.seconds // 60) % 60
    secondi = differenza.seconds % 60
    return "Il tempo totale di esecuzione e' di " + str(ore) + " ore, " + str(minuti) + " minuti e " + str(secondi) + " secondi"

<b>Configuro il file di log con le informazioni</b>

In [54]:
# Configurazione del logger
logging.basicConfig(filename='file_di_log.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

<b>Devo recuperare una lista con le informazioni dei comuni dal sito ugeo.urbistat.com</b>

In [55]:
response = requests.get("https://ugeo.urbistat.com/adminstat/it/it/demografia/dati-sintesi/milano/15/3")
soup = BeautifulSoup(response.content, 'html.parser')

div_link_comuni = soup.find_all("ul", class_="box_list box_list_l_3s")

link_comuni1 = div_link_comuni[0].find_all("a")

link_comuni2 = div_link_comuni[1].find_all("a")

link_comuni3 = div_link_comuni[2].find_all("a")

elenco_link_comuni = []

for link in link_comuni1:
    elenco_link_comuni.append(link['href'])

for link in link_comuni2:
    elenco_link_comuni.append(link['href'])

for link in link_comuni3:
    elenco_link_comuni.append(link['href'])
    
elenco_link_comuni.insert(0, elenco_link_comuni.pop(72))

<b>Inizio a prelevare le informazioni sui comuni e i relativi affitti:</b>

In [56]:
ora_inizio = dt.now()
ora_inizio_formatted = ora_inizio.strftime("%H:%M:%S")
open('file_di_log.log', 'w').close()
logging.info("L'esecuzione e' iniziata alle: " + ora_inizio_formatted + "\n\n")
numero = 0
for link in lista_link_comuni:
    
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # recupero il nome del comune
    nome = str(soup.find_all("li", attrs = {"class": "in-breadcrumbsLite__item nd-list__item"})[3].get_text()).replace('\t', '').replace('\n', '')
    
    # recupero le due strutture contenenti le vendite e affitti minimi e massimi, vendite e affitti medi
    vendita_affitto_medio = soup.find_all("p", attrs={"class": "cg-buildingPricesStats__highlighted-text"})
    vendita_affitto = soup.find_all("p", attrs={"class": "cg-buildingPricesStats__highlighted-subtext"})
    
    # salvo le due stringhe contenenti vendite e affitti minimi e massimi   
    stringa_vendite = str(vendita_affitto[0].get_text())
    stringa_affitti = str(vendita_affitto[1].get_text())
    
    # salvo nelle variabili
    
    vendita_media = float(str(re.findall(r'\d+[\.,]?\d+', str(vendita_affitto_medio[0].get_text()))[0]).replace('.', '').replace(',', '.'))
    affitto_medio = float(str(re.findall(r'\d+[\.,]?\d+', str(vendita_affitto_medio[1].get_text()))[0]).replace('.', '').replace(',', '.'))
    
    if(stringa_vendite == '-'):
        vendita_minima = 0
        vendita_massima = 0
    else:
        vendita_minima = float(re.findall(r'\d+[\.,]?\d+', stringa_vendite.replace('.', '').replace(',', '.'))[0])
        vendita_massima = float(re.findall(r'\d+[\.,]?\d+', stringa_vendite.replace('.', '').replace(',', '.'))[1])
    
    if(stringa_affitti == '-'):
        affitto_minimo = 0
        affitto_massimo = 0
    else:
        affitto_minimo = float(re.findall(r'\d+[\.,]?\d+', stringa_affitti.replace('.', '').replace(',', '.'))[0])
        affitto_massimo = float(re.findall(r'\d+[\.,]?\d+', stringa_affitti.replace('.', '').replace(',', '.'))[1])
    
    
    # recupero tutte le informazioni superficie, abitanti, densita e cap
    
    response = requests.get("https://ugeo.urbistat.com"+str(elenco_link_comuni[numero]))
    soup_other = BeautifulSoup(response.content, 'html.parser')
    
    div = soup_other.find("div", class_="table_data row")
    div_tr = div.find_all("tr")
    superficie = float(str(div_tr[4].find_all("td")[1].find("span").get_text()).replace('.','').replace(',','.'))
    densita = float(str(div_tr[5].find_all("td")[1].find("span").get_text()).replace('.','').replace(',','.'))
    popolazione = float(str(div_tr[6].find_all("td")[1].find("span").get_text()).replace('.','').replace(',','.'))
    famiglie = float(str(div_tr[7].find_all("td")[1].find("span").get_text()).replace('.','').replace(',','.'))
    maschi = float(str(div_tr[8].find_all("td")[1].find("span").get_text()).replace('.','').replace(',','.'))
    femmine = float(str(div_tr[9].find_all("td")[1].find("span").get_text()).replace('.','').replace(',','.'))
    stranieri = float(str(div_tr[10].find_all("td")[1].find("span").get_text()).replace('.','').replace(',','.'))
    eta = float(str(div_tr[11].find_all("td")[1].find("span").get_text()).replace('.','').replace(',','.'))
    #elenco_righe_table = div.find_all("tr")
    
    #popolazione = float(re.findall(r'(\d{1,3}(?:\.\d{3})*(?:,\d+)?)\s+abitanti',str(elenco_righe_table[3].find_all("td")[1].get_text()))[0].replace('.', '').replace(',', '.'))
    #superficie = float(re.findall(r'(\d{1,3}(?:\.\d{3})*(?:,\d+)?)\s+km²',str(elenco_righe_table[4].find_all("td")[1].get_text()))[0].replace('.', '').replace(',', '.'))
    #densita = float(re.findall(r'(\d{1,3}(?:\.\d{3})*(?:,\d+)?)\s+ab./km²',str(elenco_righe_table[5].find_all("td")[1].get_text()))[0].replace('.', '').replace(',', '.'))
    
    #popolazione = 0
    #superficie = 0
    #densita = 0
    #cap = int(str(elenco_righe_table[10].find_all("td")[1].find("span").get_text()))
    #cap = 0
    
    # creo un nuovo record per la tabella comuni e lo inserisco
    
    # recupero l'id massimo della tabella comuni
    cursor.execute("SELECT max(id) FROM comuni")
    
    risultato = cursor.fetchone()
    valore_max_id = risultato[0] if risultato[0] is not None else 0
    
    comuneId = valore_max_id +1
    
    # richiamo la funzione per recuperare le zone o frazioni di un comune se esistono
    haZone = soup.find("th", attrs={"class": "nd-table__heading"}).get_text() == 'Zone'

    # creo la query di insert nel db per tabella comuni
    insertQueryComuni = "INSERT INTO comuni "
    insertQueryComuni += "(nome, comune_appartenenza, comuneId, `vendita_minima(€/mese)`, `vendita_media(€/mese)`, `vendita_massima(€/mese)`, `affitto_minimo(€/mese)`, `affitto_medio(€/mese)`, `affitto_massimo(€/mese)`, popolazione, `densita_popolazione(ab./km²)`, `superficie(km²)`, famiglie, `maschi(%)`, `femmine(%)`, `stranieri(%)`, `eta_media(anni)`)"
    insertQueryComuni += "VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
              
    # stampo 2 diverse strighe nel log
    if (haZone):
        logging.info(str(numero+1) + ") Sto inserendo il comune di " + nome + " nella tabella COMUNI\n")
    else:
        logging.info(str(numero+1) + ") Sto inserendo il comune di " + nome + " nella tabella COMUNI")
     
    numero = numero + 1
    

    # faccio la insert
    cursor.execute(insertQueryComuni, 
                   (nome, nome, comuneId, vendita_minima, vendita_media, vendita_massima, affitto_minimo, affitto_medio, affitto_massimo, popolazione, densita, superficie, famiglie, maschi, femmine, stranieri, eta))

    conn.commit()
    
    # recupero i link degli affitti e le vendite
    link_vendite = soup.find_all('a', {"class": "cg-realEstateAdsCounters__link"}, href=True)[0]
    link_vendite = str(link_vendite['href'])
    link_affitto = soup.find_all('a', {"class": "cg-realEstateAdsCounters__link"}, href=True)[1]
    link_affitto = str(link_affitto['href'])

    
    # scrivo la query per affitti
    insertQueryAffitti = "INSERT INTO affitti "
    insertQueryAffitti += "(nome, comuneId, contratto, tipologia, `superficie(m²)`, locali, locali_descrizione, piani, indirizzo, latitudine, longitudine, stazioneId, `stazione_distanza(metri)`, `stazione_distanza(minuti)`, `prezzo(€/mese)`, `prezzo_minimo(€/mese)`, `prezzo_massimo(€/mese)`, box_auto, `giardino(m²)`, balcone, ascensore, fumatori, animali, url_annuncio)"
    insertQueryAffitti += "VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
    
    
    if (haZone):
        # se ha delle zone o frazioni recupero gli affitti solamente delle zone
        recupera_zone(soup, nome, insertQueryComuni, insertQueryAffitti)
    else:
        # se non le ha recupero le zone solo del comune
        recupera_affitti(link_affitto, comuneId, insertQueryAffitti)
    
      
conn.close()
ora_fine = dt.now()
ora_fine_formatted = ora_fine.strftime("%H:%M:%S")
logging.info("L'esecuzione e' terminata alle: " + ora_fine_formatted)
logging.info(stampaTempi(ora_inizio, ora_fine))